In [24]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import netCDF4 as nc
from osgeo import gdal, osr, gdalconst, ogr
import pandas as pd

gdal.UseExceptions()

In [2]:
in_path = "/home/antonv/PycharmProjects/lena/nc/e-run_v1.1.nc"

a = nc.Dataset(in_path)

print(a)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.6
    Title: E-RUN version 1.1
    Institution: ETH Zurich
    dimensions(sizes): lon(232), lat(101), time(781)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), float32 runoff(time, lat, lon)
    groups: 


In [3]:
"""
for var in a.variables.values():
    print(var)
"""
print(a['runoff'])

<class 'netCDF4._netCDF4.Variable'>
float32 runoff(time, lat, lon)
    long_name: Total runoff
    units: mm/day
    _FillValue: -9999.0
    missing_value: -9999.0
    time_statistic: accumulation
unlimited dimensions: time
current shape = (781, 101, 232)
filling on


In [4]:
print(a['lon'])
print(a['lon'][:])
print(a['lat'])
print(a['lat'][:])

<class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
unlimited dimensions: 
current shape = (232,)
filling on, default _FillValue of 9.969209968386869e+36 used
[-40.25 -39.75 -39.25 -38.75 -38.25 -37.75 -37.25 -36.75 -36.25 -35.75
 -35.25 -34.75 -34.25 -33.75 -33.25 -32.75 -32.25 -31.75 -31.25 -30.75
 -30.25 -29.75 -29.25 -28.75 -28.25 -27.75 -27.25 -26.75 -26.25 -25.75
 -25.25 -24.75 -24.25 -23.75 -23.25 -22.75 -22.25 -21.75 -21.25 -20.75
 -20.25 -19.75 -19.25 -18.75 -18.25 -17.75 -17.25 -16.75 -16.25 -15.75
 -15.25 -14.75 -14.25 -13.75 -13.25 -12.75 -12.25 -11.75 -11.25 -10.75
 -10.25  -9.75  -9.25  -8.75  -8.25  -7.75  -7.25  -6.75  -6.25  -5.75
  -5.25  -4.75  -4.25  -3.75  -3.25  -2.75  -2.25  -1.75  -1.25  -0.75
  -0.25   0.25   0.75   1.25   1.75   2.25   2.75   3.25   3.75   4.25
   4.75   5.25   5.75   6.25   6.75   7.25   7.75   8.25   8.75   9.25
   9.75  10.25  10.75  11.25  11.75  12.

In [27]:
nodata = -9999

def create_mask(raster_ds, vector_path, out_mask_path, disk_output=False):
    shp = vector_path
    data = raster_ds
    geo_transform = data.GetGeoTransform()
    proj = data.GetProjection()
    # source_layer = data.GetLayer()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * data.RasterXSize
    y_min = y_max + geo_transform[5] * data.RasterYSize
    x_res = data.RasterXSize
    y_res = data.RasterYSize
    mb_v = ogr.Open(shp)
    mb_l = mb_v.GetLayer()
    pixel_width = geo_transform[1]
    
    drv = "GTiff" if disk_output == True else "MEM"
    out_mask_path = out_mask_path if disk_output == True else ""
    target_ds = gdal.GetDriverByName(drv).Create(out_mask_path, x_res, y_res, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
    target_ds.SetProjection(proj)
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(nodata)
    band.FlushCache()
    
    gdal.RasterizeLayer(target_ds, [1], mb_l, options=["ATTRIBUTE=Id"])
    mask_arr = target_ds.GetRasterBand(1).ReadAsArray().copy()

    target_ds = None
    mask_arr = np.rot90(mask_arr,2)  # WHY??!
    mask_arr = np.flip(mask_arr,1)
    return mask_arr

def crop_variable_to_watershed(arr, ws_path, out_path=None, v=False):
    # 1) создаём массив для привязки изображения в формате GDAL GeoTransform:
    """
    GDAL GeoTransform:
    GT(0) x-coordinate of the upper-left corner of the upper-left pixel.
    GT(1) w-e pixel resolution / pixel width.
    GT(2) row rotation (typically zero).
    GT(3) y-coordinate of the upper-left corner of the upper-left pixel.
    GT(4) column rotation (typically zero).
    GT(5) n-s pixel resolution / pixel height (negative value for a north-up image).
    """
    s = arr.shape
    # print(arr.shape)
    # res = 0.5 # grid resolution in degrees
    lon_res = a['lon'][1] - a['lon'][0]
    lat_res = a['lat'][1] - a['lat'][0]

    xul = a['lon'][0]
    yul = a['lat'][-1]
    gt = [xul, lon_res, 0, yul, 0, -lat_res]  # GeoTransform готов

    # 2) вызываем драйвер GTiff и просим его создать нам новый файл нужного размера с одним растровым каналом:
    if out_path is None:
        out_path = ""
        drv = "MEM"
    else:
        drv = "GTiff"

    driver = gdal.GetDriverByName(drv)
    rows, cols = s
    no_bands = 1
    # ds = driver.Create(out_path, cols, rows, no_bands, gdal.GDT_Byte)
    ds = driver.Create(out_path, cols, rows, no_bands, gdal.GDT_Float32)

    # 3) записываем в файл массив измерений, значение "нет данных" и геотрансформ:
    ds.GetRasterBand(1).WriteArray(arr)
    ds.GetRasterBand(1).SetNoDataValue(nodata)
    ds.SetGeoTransform(gt)

    # 4) прикрепляем к файлу информацию о системе координат:
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)  # WGS84
    ds.SetProjection(srs.ExportToWkt())

    ################################################################################################

    # 5) обрезаем по водосбору:
    out_path = "runoff_%s_crop.tif" % date_str
    # crop_ds = gdal.Warp(out_path, ds, format="GTiff", cutlineDSName=ws_path, cropToCutline=True)  # does not work without cropToCutline=True
    # print(crop_ds)
    # НЕ РАБОТАЕТ ДЛЯ СЛИШКОМ МАЛЕНЬКИХ ВОДОСБОРОВ (КАК У ЛЕНЫ)

    ##################################################################################################
    # 5a) альтернативный путь для мелких водосборов: растеризуем водосбор и используем получившийся растр как маску, внутри которой считаем среднее значение:
    out_path = "runoff_mask.tif"  # will be ignored if disk_output=False
    mask_arr = create_mask(ds, ws_path, out_path, disk_output=False)
    # plt.imshow(mask_arr)

    masked_arr = np.where(mask_arr > 0, arr, nodata)
    # plt.imshow(masked_arr)
    # plt.colorbar()

    ds.GetRasterBand(1).WriteArray(masked_arr)

    m = masked_arr[masked_arr != nodata].mean()
    if v:
        print("Mean value is %.1f mm per day" % m)

    #############################################################################################
    # 6) все операции с исходной картинкой выполнялись до этого момента в оперативной памяти
    # запись на диск будет произведена только после выполнения следующих команд:
    ds.FlushCache()
    ds = None
    
    return m

def fmt_date(nc_time):
    # print(nc_time, "days")
    delta = timedelta(days=float(nc_time))
    return datetime.strptime("1950-01-01 00:00:00", "%Y-%m-%d %H:%M:%S") + delta


def show_runoff(arr, date_str):
    fig, ax = plt.subplots()
    ax.set_title(date_str)
    im = ax.imshow(arr)
    fig.colorbar(im, label="mm/day", orientation="horizontal")

In [38]:
ws_path = "/home/antonv/PycharmProjects/lena/nc/ws/fix_Lychkovo_g.geojson"
# ws_path = "/home/antonv/PycharmProjects/lena/nc/ws/fix_text.geojson"

date_list = []
mean_list = []

time_len = len(a["time"])

for i in range(0, time_len):
    # print(a['time'][i])  # days since 1950-1-1 00:00:00
    date_str = fmt_date(a['time'][i])
    
    if i % 100 == 0:
        print("Processing", date_str, "[%s/%s]" % (i, time_len))
        
    date_list.append(date_str)

    arr = a['runoff'][i, :, :]

    arr = np.rot90(arr, 2)  # WHY??!
    arr = np.flip(arr, 1)

    # show_runoff(arr, date_str)

    out_path = "runoff_%s.tif" % date_str
    # crop_variable_to_watershed(arr, ws_path, out_path=out_path)
    mean = crop_variable_to_watershed(arr, ws_path, out_path=None)
    mean_list.append(mean)

df = pd.DataFrame({"date_time": date_list, "mean_runoff_mm_day": mean_list})
display(df.head())
display(df.tail())

df.to_csv("reanalysis_runoff_test.csv", float_format='%.2f', index=False)

Processing 1950-12-16 00:00:00 [0/781]
Processing 1959-04-15 12:00:00 [100/781]
Processing 1967-08-16 00:00:00 [200/781]
Processing 1975-12-16 00:00:00 [300/781]
Processing 1984-04-15 12:00:00 [400/781]
Processing 1992-08-16 00:00:00 [500/781]
Processing 2000-12-16 00:00:00 [600/781]
Processing 2009-04-15 12:00:00 [700/781]


,date_time,mean_runoff_mm_day
0,1950-12-16 00:00:00,0.635893
1,1951-01-16 00:00:00,0.236675
2,1951-02-14 12:00:00,0.140853
3,1951-03-16 00:00:00,0.179860
4,1951-04-15 12:00:00,2.143269


,date_time,mean_runoff_mm_day
776,2015-08-16 00:00:00,0.217626
777,2015-09-15 12:00:00,0.214607
778,2015-10-16 00:00:00,0.251058
779,2015-11-15 12:00:00,0.404580
780,2015-12-16 00:00:00,0.809839
